In [1]:
import maup
import geopandas as gpd

In [2]:
STATE_CRS = "epsg:4269"

In [3]:
blocks = gpd.read_file("/home/max/git/census-process/final/ma/ma_block.shp").to_crs(STATE_CRS)

In [26]:
vtds = gpd.read_file("/home/max/git/census-process/final/ma/ma_vtd.shp").to_crs(STATE_CRS)

In [4]:
old_precincts = gpd.read_file("/home/max/git/VRA-data-products/Massachusetts/shapes/MA_pcts.shp").to_crs(STATE_CRS)

In [5]:
partial_cols = ["SEN", "PRES", "GOV", "TRE", "AG", "LTGOV", "AUD"]
election_cols = [x for x in old_precincts if any([x.startswith(y) for y in partial_cols])]
del election_cols[election_cols.index("SEND")]
election_cols

['SEN12GEWAR',
 'SEN12GSBRO',
 'PRES12GBOB',
 'PRES12GMRO',
 'SEN13GEMAR',
 'SEN13GGGOM',
 'SEN14GEMAR',
 'SEN14GHERR',
 'PRES16GHCL',
 'PRES16GDTR',
 'GOV14R',
 'GOV14D',
 'GOV14I',
 'GOV14U1',
 'GOV14U2',
 'GOV18R',
 'GOV18D',
 'SEN18D',
 'SEN18R',
 'SEN18U',
 'SEN13PEMAR',
 'SEN13PSLYN',
 'GOV14PMCOA',
 'GOV14PSGRO',
 'GOV14PDBER',
 'GOV14GCBAK',
 'GOV14GMCOA',
 'LTGOV14PSK',
 'LTGOV14PLC',
 'LTGOV14PML',
 'AG14PMHEAL',
 'AG14PWTOLM',
 'AG14GMHEAL',
 'AG14GJMILL',
 'TRE14PDGOL',
 'TRE14PBFIN',
 'TRE14PCONR',
 'TRE14GDGOL',
 'TRE14GMHEF',
 'AUD14GPBUM',
 'AUD14GSSAI',
 'PRES16PHCL',
 'PRES16PBSA',
 'GOV18PJGON',
 'GOV18PBMAS',
 'GOV18GCBAK',
 'GOV18GJGON',
 'SEN18GEWAR',
 'SEN18GGDIE',
 'LTGOV18PQP',
 'LTGOV18PJT',
 'AG18GMHEAL',
 'AG18GJMCMA',
 'TRE18GDGOL',
 'TRE18GKORR',
 'AUD18GSBUM',
 'AUD18GHBRA',
 'PRES20PJBI',
 'PRES20PBSA',
 'PRES20PEWA',
 'PRES20PMBL',
 'PRES20GJBI',
 'PRES20GDTR',
 'SEN20PEMAR',
 'SEN20PJKEN',
 'SEN20GEMAR',
 'SEN20GKOCO']

In [6]:
assignment = maup.assign(blocks, old_precincts)

/home/max/.pyenv/versions/3.9.4/lib/python3.9/site-packages/maup-0.6-py3.9.egg/maup/indexed_geometries.py:28: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', you can use 's.is_empty | s.isna()' to get back the old behaviour.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', 'GeoSeries.isna', UserWarning)
/home/max/.pyenv/versions/3.9.4/lib/python3.9/site-packages/maup-0.6-py3.9.egg/maup/indexed_geometries.py:28: UserWarning: GeoSeries.isna() previously returned True for both missing (None) and empty geometries. Now, it only returns True for missing values. Since the calling GeoSeries contains empty geometries, the result has changed compared to previous versions of GeoPandas.
Given a GeoSeries 's', y

In [7]:
weights = blocks.VAP / assignment.map(blocks.VAP.groupby(assignment).sum())

## Looking for unassigned or zero-population blocks
(this happens to be a case of zero-population blocks assigned to non-zero precincts)

In [8]:
weights.sum() == len(old_precincts)

False

In [9]:
old_precincts[(weights.groupby(assignment).sum() == 0)] # some elections have votes

,DISTRICT,NAME,Shape_Le_1,Shape_Area,TOWN,WARD,PRECINCT,SEN12GEWAR,SEN12GSBRO,PRES12GBOB,...,BCVAP17,NHPICVAP17,WCVAP17,AMINWCVA_5,ASIANWCV_5,BWCVAP17,AMINBCVA_5,2MORECVA_5,HCVAP17,geometry
181,1-15,Boston City Ward 1 Precinct 15,0.278457,0.000365,Boston,1,15,19,2,20,...,80.0,0.0,175.0,30.0,0.0,0.0,0.0,20.0,100.0,"MULTIPOLYGON (((-70.92402 42.32533, -70.92451 ..."


In [10]:
blocks[assignment == 181].VAP.sum()

0

## Prorating (the naive way)
(No checking)

In [11]:
prorated = maup.prorate(assignment, old_precincts[election_cols], weights)

In [12]:
old_precincts["SEN12GEWAR"].sum() - prorated["SEN12GEWAR"].sum()

19.0

## Prorating (with a small epsilon)

In [13]:
epsilon = 1e-9

In [14]:
blocks_adjusted = blocks["VAP"].replace(0, epsilon)

In [15]:
blocks_adjusted.sum() - blocks["VAP"].sum()

1.9478611648082733e-05

In [16]:
weights_adjusted = blocks_adjusted / assignment.map(blocks_adjusted.groupby(assignment).sum())

In [17]:
weights_adjusted.sum() == len(old_precincts)

True

In [18]:
prorated_adjusted = maup.prorate(assignment, old_precincts[election_cols], weights_adjusted)

In [19]:
# for col in election_cols:
print(abs(old_precincts[election_cols].sum() - prorated_adjusted[election_cols].sum()).sum())

4.267349140718579e-09


In [20]:
blocks[election_cols] = prorated_adjusted

## Assigning back up

In [24]:
assignment_to_vtds = maup.assign(blocks, vtds)

/home/max/.pyenv/versions/3.9.4/lib/python3.9/site-packages/maup-0.6-py3.9.egg/maup/intersections.py:42: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  geometries = geometries[geometries.area > area_cutoff]
/home/max/.pyenv/versions/3.9.4/lib/python3.9/site-packages/maup-0.6-py3.9.egg/maup/assign.py:26: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  return assign_to_max(intersections(sources, targets, area_cutoff=0).area)


In [27]:
vtds[election_cols] = blocks[election_cols].groupby(assignment).sum()

In [31]:
vtds[election_cols].sum()

SEN12GEWAR    1696346.0
SEN12GSBRO    1458048.0
PRES12GBOB    1921290.0
PRES12GMRO    1188314.0
SEN13GEMAR     645429.0
                ...    
PRES20GDTR    1167202.0
SEN20PEMAR     782694.0
SEN20PJKEN     629359.0
SEN20GEMAR    2357809.0
SEN20GKOCO    1177765.0
Length: 67, dtype: float64

In [32]:
old_precincts[election_cols].sum()

SEN12GEWAR    1696346
SEN12GSBRO    1458048
PRES12GBOB    1921290
PRES12GMRO    1188314
SEN13GEMAR     645429
               ...   
PRES20GDTR    1167202
SEN20PEMAR     782694
SEN20PJKEN     629359
SEN20GEMAR    2357809
SEN20GKOCO    1177765
Length: 67, dtype: int64

In [33]:
vtds.to_file("/home/max/git/vtd-migration/products/MA_vtd20.shp")